In [1]:
# import dependencies
!pip install mtcnn
!pip install keras_vggface
!pip install keras_applications
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline
from mtcnn.mtcnn import MTCNN
from matplotlib.patches import Rectangle
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from scipy.spatial.distance import cosine
import cv2
from PIL import Image
import tensorflow as tf
import base64

     |████████████████████████████████| 2.3MB 4.3MB/s 
     |████████████████████████████████| 51kB 2.7MB/s 


In [2]:
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  #my_string = base64.b64encode(bbox_array)
  # convert array into PIL image
  # arr=np.copy(bbox_array)
  bbox_PIL = PIL.Image.fromarray(bbox_array,'RGBA')
  # bbox_PIL= bbox_PIL.convert('RGBA')
  # #numpy array
  # bbox_PIL=np.array(bbox_PIL)
  # bbox_PIL[:,:,3] = (arr.max(axis = 2) > 0 ).astype(int) * 255
  # bbox_PIL = PIL.Image.fromarray(bbox_PIL,'RGBA')
  #bbox_PIL=bbox_PIL.putalpha(0)
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes
  
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img


In [3]:
!git clone https://github.com/Vishvambar-Panth/Surveillance.git

Cloning into 'Surveillance'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [4]:
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
detector=MTCNN()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
def extract_face(filename, required_size=(224, 224)):
  pixels=plt.imread(filename)
  results=detector.detect_faces(pixels)
  x1,y1,width,height=results[0]['box']
  x2,y2= x1 + width , y1+height
  face=pixels[y1:y2,x1:x2]
  a=np.asarray(face)
  image=Image.fromarray(a)
  image=image.resize(required_size)
  face_array = np.array(image)
  return face_array

def get_embeddings(filenames):
	# extract faces
	faces = [extract_face(f) for f in filenames]
	# convert into an array of samples
	samples = np.array(faces, 'float32')
	# prepare the face for the model, e.g. center pixels
	samples = preprocess_input(samples, version=2)
	# create a vggface model

	# perform prediction
	yhat = model.predict(samples)
	return yhat

def is_match(known_embedding, candidate_embedding, thresh=0.4):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:
		return True
		print('>MATCH (%.3f <= %.3f)' % (score, thresh))
	else:
		return False
		print('>NOT A MATCH (%.3f > %.3f)' % (score, thresh))

cascPath = "/content/Surveillance/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

                


94699520/94694792 [==============================] - 4s 0us/step


In [9]:
id_emb=get_embeddings(['/content/Surveillance/id.jpg'])
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
   
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])
    width = np.size(frame, 1)
    height = np.size(frame, 0)
   
    x = width/2
    y = height/2
    text_color = (255,0,0)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frames=np.copy(frame)
    bbox_array_rgba = np.zeros([480,640,3], dtype=np.uint8)
    #bbox_array=frame
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    if (len(faces)):
         for (x, y, w, h) in faces: 
           cv2.imwrite('face.jpg',frame)
           filenames = ['face.jpg']
           det_emb = get_embeddings(filenames)
           bbox_array=cv2.rectangle(bbox_array_rgba, (x, y), (x+w, y+h), (0, 255, 0), 2)
           

           
           if is_match(id_emb, det_emb):
             #print('Match')
             bbox_array = cv2.putText(bbox_array, "Authorization Successful!",(x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(255,255,0), 2)
           else:
             bbox_array = cv2.putText(bbox_array, "Alert, Unauthorized Person!",(x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(255,255,0), 2)
         arr=np.copy(bbox_array)
         bbox_PIL = PIL.Image.fromarray(bbox_array,'RGB')
         bbox_PIL= bbox_PIL.convert('RGBA')
          #numpy array
         bbox_PIL=np.array(bbox_PIL)
         bbox_PIL[:,:,3] = (arr.max(axis = 2) > 0 ).astype(int) * 255
         bbox_PIL = PIL.Image.fromarray(bbox_PIL,'RGBA')
         bbox_array=np.array(bbox_PIL)

         
         bbox_bytes = bbox_to_bytes(bbox_array)
         bbox = bbox_bytes
      
    else:
       bbox_array = np.zeros([480,640,4], dtype=np.uint8)
       bbox_bytes = bbox_to_bytes(bbox_array)
       bbox = bbox_bytes   

MessageError: ignored